In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import MultiLabelBinarizer
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay, roc_auc_score
from scipy.stats import randint
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import plot_tree

import warnings
warnings.filterwarnings("ignore", category=UserWarning)

%cd ../
from src import create_fake_patients


In [ ]:
num_read_codes = 512
list_of_read_codes = [str(x) for x in list(range(num_read_codes))]

#### Prepare the data

In [ ]:
cv_data = create_fake_patients.create_fake_patient_df(num_patients=9000, max_events=100, max_nodes=512)
test_data = create_fake_patients.create_fake_patient_df(num_patients=2000, max_events=100, max_nodes=512)
post_recal_data = create_fake_patients.create_fake_patient_df(num_patients=2000, max_events=100, max_nodes=512)

In [ ]:
def prep_data_for_model(data):
    """
    Take the data and one hot encode the parameters so it can be fed into the Random Forest.
    Args:
        data (DataFrame): Dataframe with indices and values to create sparse matrix.
    Returns:
        ohe_data (DataFrame): DataFrame with 0's and 1's depending on whether a Read code is present in a patients trajectory or not.
    """
    
    # Get a column which contains the sequence of events
    data["event_seq"] = "2"
    for i, row in data.iterrows():
        data["event_seq"][i] = [sublist[1] for sublist in data["indices"][i]][::-1]
        
    # Turn list of event codes to ohe columns
    mlb = MultiLabelBinarizer(sparse_output=True)

    ohe_data = data.join(
                pd.DataFrame.sparse.from_spmatrix(
                    mlb.fit_transform(data.pop('event_seq')),
                    index=data.index,
                    columns=mlb.classes_))
    
    # Change M and F to 1 and 0
    ohe_data.gender[ohe_data.gender == 'M'] = 1
    ohe_data.gender[ohe_data.gender == 'F'] = 0
    
    # adding any missing columns
    all_columns = list(range(num_read_codes))

    # Check if each column exists, and if not, fill it with zeros
    for col in all_columns:
        if col not in ohe_data.columns:
            ohe_data[col] = 0

    
    ohe_data.columns = ohe_data.columns.astype(str)
    
    return ohe_data

ohe_cv_data = prep_data_for_model(cv_data)
ohe_test_data = prep_data_for_model(test_data)
ohe_post_recal_data = prep_data_for_model(post_recal_data)
ohe_cv_data.head(5)

#### Random Forest Function

In [ ]:
def random_forest_run(X_train: pd.DataFrame, y_train: pd.DataFrame, X_test: pd.DataFrame, y_test: pd.DataFrame, 
                      X_test2: pd.DataFrame, y_test2: pd.DataFrame, input_types: str):
    y_train = y_train.replace({'none': 0, 'hip': 1})
    y_test = y_test.replace({'none': 0, 'hip': 1})
    y_test2 = y_test2.replace({'none': 0, 'hip':1})
    
    param_dist = {'n_estimators': randint(50, 500),
              'max_depth': randint(5, 20),
#               'criterion': ['gini', 'entropy'],
#               'min_samples_split': randint(2, 20),
#               'min_samples_leaf': randint(1, 20),
#               'max_features': ['auto', 'sqrt', 'log2', None],
#               'bootstrap': [True, False],
#               'min_impurity_decrease': [0.0, 0.1, 0.2]
             }


    rf = RandomForestClassifier()

    metric_scoring = ['accuracy', 'roc_auc']
    
    # Use random search to find the best hyperparameters
    rand_search = RandomizedSearchCV(rf, 
                                     param_distributions = param_dist, 
                                     n_iter=50, 
                                     cv=5,
                                     scoring = metric_scoring,
                                    return_train_score = True,
                                    refit='roc_auc')#,
                                    #error_score='raise')

    
    y_column = y_train.values
    

    y_flattened = y_column.ravel()

    # Fit the random search object to the data
    rand_search.fit(X_train, y_flattened)

    # Get the cross-validation results
    cv_results = rand_search.cv_results_
    
    mean_auc = cv_results['mean_test_roc_auc'][rand_search.best_index_]
    std_auc = cv_results['std_test_roc_auc'][rand_search.best_index_]
    mean_accuracy = cv_results['mean_test_accuracy'][rand_search.best_index_]
    std_accuracy = cv_results['std_test_accuracy'][rand_search.best_index_]
    print("Results from the cross-fold validation:")
    print(f'Mean validation AUC: {mean_auc*100:.4f} +/- {std_auc*100:.4f}')
    print(f'Mean validation Accuracy: {mean_accuracy*100:.4f}% +/- {std_accuracy*100:.4f}%')
    
    df = pd.DataFrame(rand_search.cv_results_)
    df.to_csv("temp_RF_"+input_types+"_results.csv")
    
    best_rf = rand_search.best_estimator_


    # Get the best mean accuracy and its corresponding standard deviation
    best_mean_accuracy = rand_search.cv_results_['mean_test_accuracy'][rand_search.best_index_]
    best_std_accuracy = rand_search.cv_results_['std_test_accuracy'][rand_search.best_index_]
    best_params = rand_search.best_params_

    print("Best Parameters:", best_params)
    print(f"{round(best_mean_accuracy*100, 5)}% +/- {round(best_std_accuracy*100, 5)}")

    # Generate predictions with the best model
    y_pred = best_rf.predict(X_test)
    y_pred_proba = best_rf.predict_proba(X_test)[:, 1]
    
    file_full_name_proba = 'pred_proba_and_true/RF_'+input_types+'_holdout1_proba.npy'
    with open(file_full_name_proba, 'wb') as f:
        np.save(f, y_pred_proba)

    file_full_name_true = 'pred_proba_and_true/RF_'+input_types+'_holdout1_true.npy'
    with open(file_full_name_true, 'wb') as f:
        np.save(f, y_test)
        
        
    # Generate predictions with the best model for the post-recalibration test set
    y_pred2 = best_rf.predict(X_test2)
    y_pred_proba2 = best_rf.predict_proba(X_test2)[:, 1]
    
    file_full_name_proba2 = 'pred_proba_and_true/RF_'+input_types+'_holdout2_proba.npy'
    with open(file_full_name_proba2, 'wb') as f:
        np.save(f, y_pred_proba2)

    file_full_name_true2 = 'pred_proba_and_true/RF_'+input_types+'_holdout2_true.npy'
    with open(file_full_name_true2, 'wb') as f:
        np.save(f, y_test2)
        
    


    # Extract a single decision tree from the forest (e.g., the first tree)
    tree = best_rf.estimators_[0]

    plt.figure(figsize=(20, 10))
    plot_tree(tree, feature_names=X_train.columns, class_names=ohe_cv_data['replace_type'], filled=True, 
             impurity=False, proportion=True, max_depth=4)
    # plt.savefig('figs/RF_'+input_types+'_tree.png')
    plt.show()

    
    print("Results on the holdout/test data:")
    print("Accuracy:", round(accuracy_score(y_test, y_pred)*100,4),"%")
    
    print("AUC ROC:", round(roc_auc_score(y_test, y_pred_proba),4))
    
    cm = confusion_matrix(y_test, y_pred)

    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['none', 'hip']).plot(cmap='Blues');
    

    # plt.savefig('figs/RF_'+input_types+'_confusion_matrix.png', bbox_inches='tight')

   
    return best_rf, X_train
    

#### Random Forest: All demographics and all Read Codes

In [ ]:
model_variables = ['gender', 'imd_quin', 'age_at_label_event'] + list_of_read_codes

X_cv = ohe_cv_data[model_variables]
y_cv = ohe_cv_data[['replace_type']]

X_test = ohe_test_data[model_variables]
y_test = ohe_test_data[['replace_type']]

X_test2 = ohe_post_recal_data[model_variables]
y_test2 = ohe_post_recal_data[['replace_type']]

best_rf, X_train = random_forest_run(X_cv, y_cv, X_test, y_test, X_test2, y_test2, 'demo_and_all_read')

#### Random Forest: Only Read Codes

In [ ]:
model_variables = list_of_read_codes

X_cv = ohe_cv_data[model_variables]
y_cv = ohe_cv_data[['replace_type']]

X_test = ohe_test_data[model_variables]
y_test = ohe_test_data[['replace_type']]

X_test2 = ohe_post_recal_data[model_variables]
y_test2 = ohe_post_recal_data[['replace_type']]

best_rf, X_train = random_forest_run(X_cv, y_cv, X_test, y_test, X_test2, y_test2, 'read_only')

#### Random Forest: Only demographics

In [ ]:
model_variables = ['gender', 'imd_quin', 'age_at_label_event']

X_cv = ohe_cv_data[model_variables]
y_cv = ohe_cv_data[['replace_type']]

X_test = ohe_test_data[model_variables]
y_test = ohe_test_data[['replace_type']]

X_test2 = ohe_post_recal_data[model_variables]
y_test2 = ohe_post_recal_data[['replace_type']]

best_rf, X_train = random_forest_run(X_cv, y_cv, X_test, y_test, X_test2, y_test2, 'demo_only')